<a href="https://colab.research.google.com/github/LapTQ/handwritten_text_recognition/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Open in Colab](https://colab.research.google.com/github/LapTQ/handwritten_text_recognition/blob/main/demo.ipynb)

In [1]:
!git clone https://github.com/LapTQ/handwritten_text_recognition.git
!pip install jiwer
%cd handwritten_text_recognition

Cloning into 'handwritten_text_recognition'...
remote: Enumerating objects: 2698, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 2698 (delta 72), reused 83 (delta 37), pack-reused 2579
Receiving objects: 100% (2698/2698), 426.99 MiB | 36.21 MiB/s, done.
Resolving deltas: 100% (132/132), done.
Checking out files: 100% (2403/2403), done.
     |████████████████████████████████| 50 kB 3.0 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149867 sha256=17153d2928530d63aac2c2558c616cfd36b22fe5839c7e9e5a44b2d6ded73856
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein
/content/handwritten_text_recognition


In [3]:
!python train.py --invert_color True

2022-05-09 19:54:25.690336: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Loading new model...
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_img (InputLayer)         [(None, 118, 2202,   0           []                               
                                1)]                                                               
                                                                                                  
 rescaling (Rescaling)          (None, 118, 2202, 1  0           ['input_img[0][0]']              
                                )                                                                 
                                                   